In [ ]:
import numpy as np
from numpy.random import Generator, PCG64
import matplotlib.pyplot as plt
import math
import time
from heateq import Exact, Simulateur

In [ ]:
seed = 213731490053398181466621250222036675538
rng = Generator(PCG64(seed))

In [ ]:
# Espérance exacte de la solution continue
mu_exact = Exact().mu
print(mu_exact)

In [ ]:
f = Simulateur(21, 100)
g = Simulateur(3, 20)

In [ ]:
def n_echantillon_X(n):
    return np.vstack(
        (
            rng.uniform(-math.pi, math.pi, (3, n)),
            rng.uniform(0.001, 0.009, (1, n)), 
            rng.uniform(-1., 1., (3, n))
        )
    )

In [ ]:
def rmse_estim(estimations, statistique_exacte):
    return math.sqrt(np.mean((estimations - statistique_exacte)**2))

def std_estim(estimations):
    return np.std(estimations, ddof=1)

def esperance_estim(estimations):
    return np.mean(estimations)

In [ ]:
def plot_fig_hf_vs_bf(n_samples, esp_mean_hf, std_mean_hf, rmse_mean_hf, esp_mean_bf, std_mean_bf, rmse_mean_bf, mu_exact, fr=True, eq_cost=False, savefig=None):
    fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 3.5))
    eb_hf = axs[0].errorbar(
        n_samples, esp_mean_hf, std_mean_hf,
        fmt='x', color='gray', lw=2, capsize=8, markeredgewidth=2, markersize=7,
        label='Monte Carlo'
    )
    eb_bf = axs[0].errorbar(
        n_samples, esp_mean_bf, std_mean_bf,
        fmt='xb', lw=2, capsize=8, markeredgewidth=2, markersize=7,
        label='Control variate'
    )
    hl = axs[0].axhline(y=mu_exact, linestyle="--", dashes=(3, 3), linewidth=1, color="red", dash_capstyle="round")
    axs[0].set_xscale('log')
    axs[0].set_ylim([-20,100])
    axs[0].set_xlabel( ("budget" if eq_cost else "taille de l'échantillon") if fr else ("budget" if eq_cost else "sample size") )
    axs[0].set_ylabel("estimateurs (moyenne ± ecart-type)" if fr else "estimators (mean ± std)")

    axs[1].plot(n_samples, rmse_mean_hf, 'o-', color='gray', lw=3, markeredgewidth=2, markersize=6, label='Monte Carlo HF')
    axs[1].plot(n_samples, rmse_mean_bf, 'ob-', lw=3, markeredgewidth=2, markersize=6, label='Monte Carlo BF')
    axs[1].set_xscale('log')
    axs[1].set_yscale('log')
    axs[1].set_xlabel( ("budget" if eq_cost else "taille de l'échantillon") if fr else ("budget" if eq_cost else "sample size") )
    axs[1].set_ylabel("REQM des estimateurs" if fr else "RMSE of the estimators")
    axs[1].legend()

    if savefig is not None:
        plt.savefig(savefig, bbox_inches='tight', pad_inches=0.02)


In [ ]:
# Espérance exacte de Y = f(X) (solution discrétisée, haute fidélité)
mu_Y_exact = f.mu

# Espérance exacte de Z = g(X) (solution discrétisée, basse fidélité)
mu_Z_exact = g.mu

print(mu_Y_exact, mu_Z_exact)

In [ ]:
print(mu_Y_exact - mu_exact)
print(mu_Z_exact - mu_exact)

In [ ]:
ns = 1000000
X = n_echantillon_X(ns)

st = time.process_time()
Y = f(X)
et = time.process_time()
time_f = et - st

st = time.process_time()
Z = g(X)
et = time.process_time()
time_g = et - st

w = time_g / time_f

print(time_f, time_g)
print(w, 1/35)

In [ ]:
w = 1./35.

In [ ]:
nr = 1000
n_samples = [5, 10, 20, 50, 100, 200, 500, 1000, 2000, 5000, 10000]
esp_Y_mean  = []
std_Y_mean  = []
rmse_Y_mean = []
esp_Z_mean  = []
std_Z_mean  = []
rmse_Z_mean = []
esp_Z_mean_eq_cost  = []
std_Z_mean_eq_cost  = []
rmse_Z_mean_eq_cost = []
for ns in n_samples:
    ns_eq_cost = int(ns / w)
    Y_mean = np.zeros(nr)
    Z_mean = np.zeros(nr)
    Z_mean_eq_cost = np.zeros(nr)
    for ir in range(nr):
        X = n_echantillon_X(ns)
        X_eq_cost = n_echantillon_X(ns_eq_cost)
        Y = f(X)
        Z = g(X)
        Z_eq_cost = g(X_eq_cost)
        Y_mean[ir] = np.mean(Y)
        Z_mean[ir] = np.mean(Z)
        Z_mean_eq_cost[ir] = np.mean(Z_eq_cost)
    
    esp_Y_mean  += [esperance_estim(Y_mean)]
    std_Y_mean  += [std_estim(Y_mean)]
    rmse_Y_mean += [rmse_estim(Y_mean, mu_exact)]
    
    esp_Z_mean  += [esperance_estim(Z_mean)]
    std_Z_mean  += [std_estim(Z_mean)]
    rmse_Z_mean += [rmse_estim(Z_mean, mu_exact)]

    esp_Z_mean_eq_cost  += [esperance_estim(Z_mean_eq_cost)]
    std_Z_mean_eq_cost  += [std_estim(Z_mean_eq_cost)]
    rmse_Z_mean_eq_cost += [rmse_estim(Z_mean_eq_cost, mu_exact)]
    
    print("HF", ns, ns,
          np.mean(Y_mean), rmse_estim(Y_mean, mu_exact), std_estim(Y_mean)
         )
    print("BF", ns, ns_eq_cost,
          np.mean(Z_mean), rmse_estim(Z_mean, mu_exact), std_estim(Z_mean),           
          np.mean(Z_mean_eq_cost), rmse_estim(Z_mean_eq_cost, mu_exact), std_estim(Z_mean_eq_cost)
         )
    print()

In [ ]:
plot_fig_hf_vs_bf(n_samples, esp_Y_mean, std_Y_mean, rmse_Y_mean, esp_Z_mean, std_Z_mean, rmse_Z_mean, mu_exact, fr=True, savefig="fig_hf_bf.pdf")

In [ ]:
plot_fig_hf_vs_bf(n_samples, esp_Y_mean, std_Y_mean, rmse_Y_mean, esp_Z_mean_eq_cost, std_Z_mean_eq_cost, rmse_Z_mean_eq_cost, mu_exact, fr=True, eq_cost=True, savefig="fig_hf_bf_eq_cost.pdf")